## Подключение к метастору и проверка работы

In [1]:
import duckdb
import os

def env(name, default=None):
    v = os.environ.get(name)
    return v if v not in (None, "") else default

def install_extensions(conn: duckdb.DuckDBPyConnection) -> None:
    conn.execute("""
        INSTALL ducklake;
        INSTALL postgres_scanner;
        INSTALL httpfs;
    """)

def chech_extensions(conn: duckdb.DuckDBPyConnection) -> None:
    print("Extensions:")
    conn.sql("""
        select extension_name, loaded, installed, description, aliases
        from duckdb_extensions()
        where 1 = 1
            and extension_name in ('ducklake', 'httpfs', 'postgres_scanner')
    """).show()


def create_secrets(conn: duckdb.DuckDBPyConnection) -> None:
    conn.execute(f"""
    CREATE OR REPLACE SECRET minio_storage (
        TYPE S3,
        KEY_ID '{env("MINIO_ROOT_USER")}',
        SECRET '{env("MINIO_ROOT_PASSWORD")}',
        ENDPOINT '{env("MINIO_ENDPOINT")}',
        REGION 'eu-central-1',
        USE_SSL false,
        URL_STYLE 'path',
        SCOPE 's3://{env("MINIO_BUCKET")}/'
    );
    """)

    conn.execute(f"""
    CREATE OR REPLACE SECRET pg_meta (
        TYPE POSTGRES,
        HOST '{env("POSTGRES_HOST")}',
        PORT {env("POSTGRES_PORT")},
        DATABASE '{env("POSTGRES_DB")}',
        USER '{env("POSTGRES_USER")}',
        PASSWORD '{env("POSTGRES_PASSWORD")}'
    );
    """)

    conn.execute(f"""
    ATTACH 'ducklake:postgres:' AS lake (
        META_SECRET pg_meta,
        DATA_PATH 's3://{env("MINIO_BUCKET")}/'
    );
    """)
    
    conn.execute("USE lake;")

with duckdb.connect() as conn:
    install_extensions(conn)
    create_secrets(conn)

    conn.sql('from duckdb_schemas()').show()

┌───────┬──────────────────────────┬──────────────┬────────────────────┬─────────┬───────────────────────┬──────────┬─────────┐
│  oid  │      database_name       │ database_oid │    schema_name     │ comment │         tags          │ internal │   sql   │
│ int64 │         varchar          │    int64     │      varchar       │ varchar │ map(varchar, varchar) │ boolean  │ varchar │
├───────┼──────────────────────────┼──────────────┼────────────────────┼─────────┼───────────────────────┼──────────┼─────────┤
│  2059 │ __ducklake_metadata_lake │         2055 │ information_schema │ NULL    │ {}                    │ true     │ NULL    │
│  2056 │ __ducklake_metadata_lake │         2055 │ pg_catalog         │ NULL    │ {}                    │ true     │ NULL    │
│  2062 │ __ducklake_metadata_lake │         2055 │ pg_temp_3          │ NULL    │ {}                    │ true     │ NULL    │
│  2064 │ __ducklake_metadata_lake │         2055 │ pg_temp_4          │ NULL    │ {}                   

### Аналитическая часть

Данные:
- `browser_events.jsonl` — данные с информацией о просмотрах браузера (у клика много id)
    
    Здесь приходит timestamp (UTC), тип события, и два уникальных поля `event_id` и `click_id`, которые используются для склеивания с другими данными. Пример записи:
    ```json
    {
      "event_id": "8cca1c7d-b0cc-4738-be92-c644101e3fff",
      "event_timestamp": "2022-11-28 20:51:05.627882",
      "event_type": "pageview",
      "click_id": "811320f1-3bc2-42b9-a841-5a1e5a812f2d",
      "browser_name": "Chrome",
      "browser_user_agent": "Mozilla/5.0 (Linux; Android 2.3.5) AppleWebKit/531.2 (KHTML, like Gecko) Chrome/32.0.833.0 Safari/531.2",
      "browser_language": "sat_IN"
    }
    ```

- `device_events.jsonl` — данные об устройствах, с которых пользователи пользовались сайтом

    Отсюда можно собрать данные об операционной системе и типе устройства. Склеить их с остальными данными можно по полю `click_id`. Также в этих данных есть информация о пользователе. Пример записи (судя по операционной системе перед нами путешественник из прошлого):
    ```json
    {
      "click_id": "811320f1-3bc2-42b9-a841-5a1e5a812f2d",
      "os": "iPad; CPU iPad OS 4_2_1 like Mac OS X",
      "os_name": "iOS",
      "os_timezone": "Europe/Berlin",
      "device_type": "Mobile",
      "device_is_mobile": true,
      "user_custom_id": "aperry@yahoo.com",
      "user_domain_id": "1ab06c9f-0e2e-4f46-9b6c-91a0e9a86a4d"
    }
    ```

- `geo_events.jsonl` — данные о локации пользователя

    Вам известно, что тут вам в базовом наборе дали только неточные данные, которые уже все события привязывают к определённому городу. Склеивать с остальными наборами данных нужно по полю `click_id`. Пример записи:
    ```json
    {
      "click_id": "811320f1-3bc2-42b9-a841-5a1e5a812f2d",
      "geo_latitude": "50.82709",
      "geo_longitude": "6.9747",
      "geo_country": "DE",
      "geo_timezone": "Europe/Berlin",
      "geo_region_name": "Wesseling",
      "ip_address": "206.227.30.186"
    }
    ```
    
- `location_events.jsonl` — вопреки названию, это данные не о локации пользователя, а непосредственно о положении пользователя на сайте и информации о том, откуда пользователь попал на страницу

    В примере, который вам дали, явно фейковые данные, которые не соответствуют настоящему сайту, но поставщик данных уверяет, что схема данных в реальной системе такая же. Склеивать с остальными данными можно по полю `event_id`. Помимо непосредственно событий непосредственно посещения сайта, тут также маркетинговые метки. Пример записи:
    ```json
    {
      "event_id": "8cca1c7d-b0cc-4738-be92-c644101e3fff",
      "page_url": "http://www.dummywebsite.com/home",
      "page_url_path": "/home",
      "referer_url": "www.facebook.com",
      "referer_medium": "internal",
      "utm_medium": "organic",
      "utm_source": "facebook",
      "utm_content": "ad_4",
      "utm_campaign": "campaign_2"
    }
    ```

Далее мы считаем покупки как переход по ссылке `/product-*` и оплата в момент `confirmation`, `click_id` - id сессии пользователя  до сброса куков.

Возможные пути по сайту:
- `/cart` - корзина
- `/home` - домашняя страница
- `/payment` - оплата
- `/confirmation` - подтверждение оплаты
- `/product_*` - продукт

In [51]:
with duckdb.connect() as conn:
    create_secrets(conn)

    conn.sql("""
        select be.click_id, be.event_id, be.event_timestamp, be.browser_name,
            le.page_url_path, le.utm_medium, le.utm_source, le.utm_content, le.utm_campaign
        from raw.browser_events be
            join raw.location_events le on be.event_id = le.event_id
        where be.click_id = '00090688-6ff7-46e0-85e6-b403a5af0672'
        order by be.event_timestamp
    """).show(max_rows=50)

┌──────────────────────────────────────┬──────────────────────────────────────┬─────────────────────────┬──────────────┬───────────────────────────────────┬────────────┬────────────┬─────────────┬──────────────┐
│               click_id               │               event_id               │     event_timestamp     │ browser_name │           page_url_path           │ utm_medium │ utm_source │ utm_content │ utm_campaign │
│                 uuid                 │                 uuid                 │        timestamp        │   varchar    │              varchar              │  varchar   │  varchar   │   varchar   │   varchar    │
├──────────────────────────────────────┼──────────────────────────────────────┼─────────────────────────┼──────────────┼───────────────────────────────────┼────────────┼────────────┼─────────────┼──────────────┤
│ 00090688-6ff7-46e0-85e6-b403a5af0672 │ 8ea57809-e21c-46c2-884f-a5f5c6786f49 │ 2025-11-27 08:06:07.296 │ Firefox      │ /product_galstuk               

In [3]:
with duckdb.connect() as conn:
    install_extensions(conn)
    create_secrets(conn)

    conn.sql("""
        with t as (
            select ge.ip_address, ge.click_id
                , de.user_domain_id, de.user_custom_id
                --, be.event_timestamp
                , count(*) cnt
            from raw.geo_events ge 
                join raw.device_events de on ge.click_id = de.click_id
                join raw.browser_events be on ge.click_id = be.click_id
            group by 1, 2, 3, 4
        )

        select ip_address, count(*) cnt
        from t
        group by 1
        order by cnt desc
    """).show(max_rows=5)

┌────────────────┬───────┐
│   ip_address   │  cnt  │
│    varchar     │ int64 │
├────────────────┼───────┤
│ 212.238.85.15  │     2 │
│ 201.62.230.122 │     2 │
│ 100.215.75.2   │     2 │
│      ·         │     · │
│      ·         │     · │
│      ·         │     · │
│ 3.179.242.17   │     1 │
│ 122.31.190.187 │     1 │
├────────────────┴───────┤
│ ? rows       2 columns │
│ (>9999 rows, 5 shown)  │
└────────────────────────┘



In [7]:
with duckdb.connect() as conn:
    install_extensions(conn)
    create_secrets(conn)

    conn.sql("""
        select ge.ip_address, ge.click_id, 
            de.user_domain_id, de.user_custom_id, de.device_type,
            count(*) cnt
        from raw.geo_events ge 
            join raw.device_events de on ge.click_id = de.click_id
            join raw.browser_events be on ge.click_id = be.click_id
        where ge.ip_address = '212.238.85.15'
        group by 1, 2, 3, 4, 5
    """).show(max_rows=5)

┌───────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────┬─────────────┬───────┐
│  ip_address   │               click_id               │            user_domain_id            │      user_custom_id      │ device_type │  cnt  │
│    varchar    │                 uuid                 │               varchar                │         varchar          │   varchar   │ int64 │
├───────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────┼─────────────┼───────┤
│ 212.238.85.15 │ 519437b9-1ea5-4c73-a74e-4a1436d0e754 │ bcc238f6-b4a9-464e-96ba-38765d15c09e │ ritamorgagni@yahoo.com   │ Mobile      │     1 │
│ 212.238.85.15 │ 83e7ea2f-854d-4c12-9013-2e69f07c9543 │ d6af7288-ecbb-4709-bfcb-39cc990125d3 │ zoppettimatteo@libero.it │ Computer    │    64 │
└───────────────┴──────────────────────────────────────┴──────────────────────────────────────┴──────────────────────────┴────────

## Stage


In [71]:
with duckdb.connect() as conn:
    install_extensions(conn)
    create_secrets(conn)

    conn.execute("create schema if not exists stage;")

Продажи по сессиям

In [76]:
with duckdb.connect() as conn:
    install_extensions(conn)
    create_secrets(conn)

    # purchases
    conn.execute("""
        create table stage.purchases as
        with events AS (
            select 
                be.click_id,
                be.event_id,
                be.event_timestamp,
                le.page_url_path,
                case when le.page_url_path LIKE '/product_%' then 1 else 0 end is_product,
                case when le.page_url_path = '/confirmation' then 1 else 0 end is_confirmation
            from raw.browser_events be
            join raw.location_events le ON be.event_id = le.event_id
        ),
        confirmation_events AS (
            select click_id, event_id, event_timestamp
            from events
            WHERE is_confirmation = 1
        ),
        product_counts AS (
            select ce.event_id, ce.event_timestamp, count(*) product_count
            from confirmation_events ce
            left join events pe on pe.click_id = ce.click_id
                and pe.is_product = 1 
                and pe.event_timestamp < ce.event_timestamp
            GROUP BY ce.event_id, ce.event_timestamp
        ), cumulative as (
            select 
                be.click_id,
                be.event_id,
                be.event_timestamp,
                COALESCE(pc.product_count, 0) product_count
            from raw.browser_events be
            join raw.location_events le on be.event_id = le.event_id
            left join product_counts pc on be.event_id = pc.event_id
            where le.page_url_path = '/confirmation'
        ), result as (
            select c1.click_id session_id, c1.event_timestamp,
                c1.product_count - coalesce((
                    select max(c2.product_count) 
                    from cumulative c2 
                    where c2.event_timestamp < c1.event_timestamp 
                        and c1.click_id = c2.click_id), 0) product_count
            from cumulative c1
            order by c1.click_id, c1.event_timestamp
        )
        
        select *
        from result
    """)

Информация по сессиям

In [79]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # session_info
    conn.execute("""
        create table stage.session_info as
        select distinct click_id session_id, browser_name, browser_user_agent, browser_language
        from raw.browser_events
    """)

AttributeError: 'NoneType' object has no attribute 'show'

In [2]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # session_info
    conn.sql("""
        select *
        from stage.session_info
    """).show()

┌──────────────────────────────────────┬───────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────┐
│              session_id              │ browser_name  │                                                                 browser_user_agent                                                                 │ browser_language │
│                 uuid                 │    varchar    │                                                                      varchar                                                                       │     varchar      │
├──────────────────────────────────────┼───────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┤
│ 9814905c-4a0d-4d0f-af46-83e5b9bbbf96 │ Chrome        │ Mozilla/5.0 (Macintosh; Intel Mac OS X 10_1

Информация по устройствам (реплика)

In [80]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # devices
    conn.execute("""
        create table stage.devices as
        select distinct click_id session_id, os_name, os_timezone, device_type, 
            user_custom_id, user_domain_id
        from raw.device_events
    """)

Информация о локации пользователя

In [81]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # geo
    conn.execute("""
        create table stage.geo as
        select distinct click_id session_id, geo_latitude, geo_longitude, geo_country, geo_timezone, 
            geo_region_name, ip_address
        from raw.geo_events
    """)

Информация о трафике

In [93]:
with duckdb.connect() as conn:
    create_secrets(conn)

    conn.execute("drop table stage.transitions")
    # transitions
    conn.execute("""
        create table stage.transitions as
        select le.event_id, le.page_url, le.page_url_path, be.event_timestamp
        from raw.location_events le
            join raw.browser_events be on le.event_id = be.event_id
    """)

Привязка сесиии к ивенту

In [83]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # events
    conn.execute("""
        create table stage.events as
        select distinct click_id session_id, event_id, event_timestamp
        from raw.browser_events
    """)

Ссылки для перехода на сайт

In [88]:
with duckdb.connect() as conn:
    create_secrets(conn)
    
    # conversion
    conn.execute("""
        create table stage.conversion as
        with merge as (
            select be.click_id session_id, max(le.event_id) event_id
            from raw.location_events le
                join raw.browser_events be on be.event_id = le.event_id
            group by 1
        )
        select m.session_id, le.referer_url, le.referer_medium, le.utm_medium, le.utm_source, 
            le.utm_content, le.utm_campaign
        from merge m
            join raw.location_events le on le.event_id = m.event_id
        order by session_id
    """)

## Mart

In [107]:
with duckdb.connect() as conn:
    install_extensions(conn)
    create_secrets(conn)

    conn.execute("create schema if not exists mart;")

Распределение событий по часам

In [108]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # events_distribution_hourly
    conn.execute("""
        create table mart.events_distribution_hourly as
        with t as (
            select event_id, 
                case 
                    when page_url_path like '/product_%' then 'add product'
                    when page_url_path = '/home' then 'go home'
                    when page_url_path = '/payment' then 'payment cart'
                    when page_url_path = '/cart' then 'go cart'
                    when page_url_path = '/confirmation' then 'confirmation payment'
                end as event, event_timestamp
            from stage.transitions
        )
        select event, date_trunc('hour', event_timestamp) date_part, count(*) cnt
        from t
        group by 1, 2
    """)

In [136]:
with duckdb.connect() as conn:
    create_secrets(conn)
    conn.sql("select * from mart.events_distribution_hourly").show()

┌──────────────────────┬─────────────────────┬───────┐
│        event         │      date_part      │  cnt  │
│       varchar        │      timestamp      │ int64 │
├──────────────────────┼─────────────────────┼───────┤
│ confirmation payment │ 2025-11-19 20:00:00 │  1389 │
│ add product          │ 2025-11-15 13:00:00 │  8169 │
│ go home              │ 2025-11-20 06:00:00 │  1733 │
│ add product          │ 2025-11-23 19:00:00 │  8676 │
│ go home              │ 2025-11-25 13:00:00 │  5655 │
│ add product          │ 2025-11-29 14:00:00 │  7476 │
│ payment cart         │ 2025-11-25 05:00:00 │   300 │
│ confirmation payment │ 2025-11-28 00:00:00 │   580 │
│ add product          │ 2025-11-24 21:00:00 │  5440 │
│ add product          │ 2025-11-29 16:00:00 │  7676 │
│      ·               │          ·          │    ·  │
│      ·               │          ·          │    ·  │
│      ·               │          ·          │    ·  │
│ add product          │ 2025-11-30 03:00:00 │  1381 │
│ go home 

Количество купленных товаров

In [111]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # purchased_products_count_hourly
    conn.execute("""
        create table mart.purchased_products_count_hourly as
        select date_trunc('hour', event_timestamp) date_part, sum(product_count) cnt
        from stage.purchases
        group by 1
    """)

In [137]:
with duckdb.connect() as conn:
    create_secrets(conn)
    conn.sql("select * from mart.purchased_products_count_hourly").show()

┌─────────────────────┬────────┐
│      date_part      │  cnt   │
│      timestamp      │ int128 │
├─────────────────────┼────────┤
│ 2025-11-21 10:00:00 │   1740 │
│ 2025-11-24 20:00:00 │   2291 │
│ 2025-11-23 09:00:00 │   1353 │
│ 2025-11-18 17:00:00 │   2209 │
│ 2025-11-28 21:00:00 │   1362 │
│ 2025-11-21 15:00:00 │   1719 │
│ 2025-11-17 06:00:00 │    554 │
│ 2025-11-15 21:00:00 │   1300 │
│ 2025-11-29 08:00:00 │   1155 │
│ 2025-11-22 14:00:00 │   1880 │
│          ·          │     ·  │
│          ·          │     ·  │
│          ·          │     ·  │
│ 2025-11-20 09:00:00 │   1310 │
│ 2025-11-20 11:00:00 │   1947 │
│ 2025-11-15 19:00:00 │   2363 │
│ 2025-11-30 00:00:00 │    853 │
│ 2025-11-19 12:00:00 │   2274 │
│ 2025-11-23 05:00:00 │    316 │
│ 2025-11-19 10:00:00 │   1797 │
│ 2025-11-21 16:00:00 │   1989 │
│ 2025-11-25 15:00:00 │   1627 │
│ 2025-11-21 06:00:00 │    515 │
├─────────────────────┴────────┤
│ 363 rows (20 shown)2 columns │
└──────────────────────────────┘



Список ссылок-переходов с количеством покупок

In [120]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # click_conversion_rates_hourly
    conn.execute("""
        create table mart.click_conversion_rates_hourly as
        select c.utm_source, date_trunc('hour', p.event_timestamp) date_part, sum(p.product_count) sm
        from stage.purchases p
            join stage.conversion c on p.session_id = c.session_id
        group by 1, 2
    """)

In [138]:
with duckdb.connect() as conn:
    create_secrets(conn)
    conn.sql("select * from mart.click_conversion_rates_hourly").show()

┌────────────┬─────────────────────┬────────┐
│ utm_source │      date_part      │   sm   │
│  varchar   │      timestamp      │ int128 │
├────────────┼─────────────────────┼────────┤
│ google     │ 2025-11-18 17:00:00 │    261 │
│ google     │ 2025-11-17 16:00:00 │    342 │
│ dzen       │ 2025-11-16 15:00:00 │    315 │
│ telegram   │ 2025-11-22 07:00:00 │    248 │
│ mailchimp  │ 2025-11-24 13:00:00 │    643 │
│ mailchimp  │ 2025-11-16 13:00:00 │    667 │
│ telegram   │ 2025-11-22 10:00:00 │    568 │
│ telegram   │ 2025-11-22 16:00:00 │    538 │
│ mailchimp  │ 2025-11-19 20:00:00 │    778 │
│ telegram   │ 2025-11-16 12:00:00 │    585 │
│   ·        │          ·          │      · │
│   ·        │          ·          │      · │
│   ·        │          ·          │      · │
│ yandex     │ 2025-11-28 09:00:00 │     77 │
│ google     │ 2025-11-18 04:00:00 │     27 │
│ dzen       │ 2025-11-26 02:00:00 │     67 │
│ yandex     │ 2025-11-20 14:00:00 │     79 │
│ yandex     │ 2025-11-17 18:00:00

Анализ покупок по рекламным компаниям

In [123]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # campaign_purchase_analysis_hourly
    conn.execute("""
        create table mart.campaign_purchase_analysis_hourly as
        select c.utm_campaign, date_trunc('hour', p.event_timestamp) date_part, sum(p.product_count) sm
        from stage.purchases p
            join stage.conversion c on p.session_id = c.session_id
        group by 1, 2
    """)

In [139]:
with duckdb.connect() as conn:
    create_secrets(conn)
    conn.sql("select * from mart.campaign_purchase_analysis_hourly").show()

┌──────────────┬─────────────────────┬────────┐
│ utm_campaign │      date_part      │   sm   │
│   varchar    │      timestamp      │ int128 │
├──────────────┼─────────────────────┼────────┤
│ campaign_1   │ 2025-11-28 16:00:00 │   1508 │
│ campaign_1   │ 2025-11-25 09:00:00 │   1177 │
│ campaign_2   │ 2025-11-24 17:00:00 │    378 │
│ campaign_1   │ 2025-11-15 20:00:00 │   1902 │
│ campaign_1   │ 2025-11-21 20:00:00 │   1892 │
│ campaign_1   │ 2025-11-23 08:00:00 │   1031 │
│ campaign_1   │ 2025-11-29 18:00:00 │   1933 │
│ campaign_1   │ 2025-11-20 12:00:00 │   1734 │
│ campaign_1   │ 2025-11-17 13:00:00 │   1670 │
│ campaign_2   │ 2025-11-16 07:00:00 │    178 │
│     ·        │          ·          │     ·  │
│     ·        │          ·          │     ·  │
│     ·        │          ·          │     ·  │
│ campaign_2   │ 2025-11-25 23:00:00 │    185 │
│ campaign_2   │ 2025-11-19 01:00:00 │     95 │
│ campaign_2   │ 2025-11-23 08:00:00 │    264 │
│ campaign_2   │ 2025-11-24 01:00:00 │  

Процентное соотношение пользователей в различных сегментах

In [ ]:
# user_segments_percentage

Сегментация графиков

In [ ]:
# funnel_segmentation_analysis

Топ товаров

In [135]:
with duckdb.connect() as conn:
    create_secrets(conn)

    # top_performing_products
    conn.execute("""
        create table mart.top_performing_products as
        select substring(page_url_path, strpos(page_url_path, '_') + 1) product, count(*) cnt
        from stage.transitions  
        where page_url_path like '/product_%'
        group by 1
    """)

In [140]:
with duckdb.connect() as conn:
    create_secrets(conn)
    conn.sql("select * from mart.top_performing_products").show()

┌──────────────────────────┬────────┐
│         product          │  cnt   │
│         varchar          │ int64  │
├──────────────────────────┼────────┤
│ kedy                     │  63522 │
│ noski                    │  32000 │
│ flower_pot               │ 151902 │
│ poster                   │  15931 │
│ hoodie                   │ 246382 │
│ cheese_cutter            │  48284 │
│ fish_stick               │  31735 │
│ fancy_dress              │  31924 │
│ kruzhka                  │ 226710 │
│ termos                   │ 229964 │
│ microphone               │  31839 │
│ sertifikat_zapreta_kinzy │ 239013 │
│ neponyatnaya_hrenovina   │  20134 │
│ book                     │  16887 │
│ secret_shtuchka          │  16007 │
│ fake_broccoli            │  16033 │
│ tolstovka                │ 238858 │
│ coffee                   │  53820 │
│ flaska                   │ 152092 │
│ galstuk                  │ 110919 │
├──────────────────────────┴────────┤
│ 20 rows                 2 columns │
└───────────